# Partie 1

## Modules

In [ ]:
import openpyxl
from read_input import read_ressources,read_ressources_unavailabilities,read_tasks,read_tasks_unavailabilities

from gurobipy import *
import math

## Lecture des données

In [ ]:
ressources = read_ressources(1)
ressources_unavailabilities = read_ressources_unavailabilities(1)
tasks = read_tasks(1)
tasks_unavailabilities = read_tasks_unavailabilities(1)

## Structures de données

In [ ]:
def dist(lat1, long1, lat2, long2):
    return(math.sqrt((lat1-lat2)**2+(long1-long2)**2))

v=833.333333333

L_ressources = [j for j in ressources]
L_tasks = [i for i in tasks]
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (task.latitude, task.longitude) for task in tasks.values()}

dict_pos = {task : (tasks[L_tasks[task]].latitude, tasks[L_tasks[task]].longitude) for task in range(number_of_tasks)}
#graph = {(task1,task2): dist(float(tasks[L_tasks[task1].latitude]), float(tasks[L_tasks[task1].longitude]), float(tasks[L_tasks[task2].latitude]), float(tasks[L_tasks[task2].longitude])) for task1 in range(number_of_tasks) for task2 in range(number_of_tasks)}
print(dict_pos)

#distance[a][b] donne la distance de la tâche a vers la tâche b
distances = [[dist(tasks[L_tasks[a]].latitude,tasks[L_tasks[a]].longitude, tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude) for b in range(number_of_tasks)] for a in range(number_of_tasks)]

# creer locations
# creer dists

# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html

## Variables de décision

In [ ]:
### Create Model
m = Model("trs0")

### Decision variables
# Task-ressource assignment
x = {(t,r) : m.addVar(vtype = GRB.BINARY, name = f'x_{t}_{r}') for r in range(number_of_ressources) for t in range(number_of_tasks)}

# Time-task assignment
s = {t : m.addVar(vtype = GRB.INTEGER, lb = 0, ub = 1440, name = f's_{t}') for t in range(number_of_tasks)}
# Les s_t valent entre 0 et 1440 car les contraintes de temps (entre 8h et 18h) sont contenues dans les jeux de données.

# Est-ce que le travailleur se déplace entre les points a et b ?
y = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
# Technician cannot leave or return to a depot that is not its base

# Is the worker used ?
u = {r: m.addVar(vtype = GRB.BINARY, name = f'u_{r}') for r in range(number_of_ressources)}

# starting task of worker
d = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'd_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks)}
# ending task of worker
f = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'f_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks)}

#variables de linéarisation
H = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'h_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
P = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'p_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}


## Contraintes

In [ ]:
C4_2 = [m.addConstr(y[(r,a,a)] == 0) for r in range(number_of_ressources) for a in range(number_of_tasks)]

In [ ]:
nbRpT = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpT[t]+=x[(t,r)]

# One ressource per task
C1 = [m.addConstr(nbRpT[t] == 1) for t in range(number_of_tasks)]

# Works only if all tasks are feasable
#nbRpTq = {t: LinExpr() for t in range(number_of_tasks)}
#for t in range(number_of_tasks):
#  for r in  range(number_of_ressources):
#    if ressources[L_ressources[r]].skill==tasks[L_tasks[t]].skill and ressources[L_ressources[r]].level>=tasks[L_tasks[t]].level:
#      nbRpTq[t]+=x[(t,r)]

# Qualified technicians
#C2 = [m.addConstr(nbRpTq[t] >= 1) for t in range(number_of_tasks)]


In [ ]:
nbRpTs = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
     nbRpTs[t]+=x[(t,r)]*(ressources[L_ressources[r]].skill!=tasks[L_tasks[r]].skill)

 # Skilled technicians
C2 = [m.addConstr(nbRpTs[t] == 0) for t in range(number_of_tasks)]

In [ ]:
nbRpTl =[ LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpTl[t]+=x[(t,r)]*(ressources[L_ressources[r]].level-tasks[L_tasks[r]].level)

# Leveled technicians
C3 = [m.addConstr(nbRpTl[t] >= 0) for t in range(number_of_tasks)]

In [ ]:
# si une personne est affectée, elle arrive jusqu'à la tâche
C4 = {(i,j) : m.addConstr(quicksum(y[(j,a,i)] for a in range(number_of_tasks)) == x[(i,j)], name = f'arrive{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

# une personne doit partir après avoir exécuté sa tâche
C5 = {(i,j) : m.addConstr(quicksum(y[(j,i,b)] for b in range(number_of_tasks)) == x[(i,j)], name = f'part{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

In [ ]:
# Non overlapping tasks

C6 = [m.addConstr(H[(r,a,b)] >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
C7 = [m.addConstr(H[(r,a,b)] - 1440 * y[(r,a,b)]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
C8 = [m.addConstr(H[(r,a,b)] - s[a]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
C9 = [m.addConstr(H[(r,a,b)] - s[a] + 1440*(1-H[(r,a,b)])  >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]

C10 = [m.addConstr(P[(r,a,b)] >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
C11 = [m.addConstr(P[(r,a,b)] - 1440 * y[(r,a,b)]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
C12 = [m.addConstr(P[(r,a,b)] - s[b]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
C13 = [m.addConstr(P[(r,a,b)] - s[b] + 1440*(1-P[(r,a,b)])  >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
C14Sum =[[LinExpr() for a in range(number_of_tasks)] for b in range(number_of_tasks)]
for a in range(number_of_tasks):
  for b in range(number_of_tasks):
    for j in  range(number_of_ressources):
      C14Sum[a][b]+=H[(j,a,b)] - P[(j,a,b)] + y[(j,a,b)] * (tasks[L_tasks[a]].duration + distances[a][b]/v)

C14 = [m.addConstr(C14Sum[a][b] <= 0) for a in range(number_of_tasks) for b in range(number_of_tasks)]

In [ ]:
M=100000

C7 = [m.addConstr(quicksum(d[(j,a)] for a in range(number_of_tasks)) == u[j]) for j in range(number_of_ressources)]
C8 = [m.addConstr(quicksum(f[(j,b)] for b in range(number_of_tasks)) == u[j]) for j in range(number_of_ressources)]

C9 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) <= u[j]*M) for j in range(number_of_ressources)]
C10 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) >= u[j]) for j in range(number_of_ressources)]

C13 = [d[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]
C14 = [f[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]



## Fonction objectif et résolution

In [ ]:
# -- Ajout de la fonction objectif --
couts=LinExpr()
for j in range(number_of_ressources):
  for a in range(number_of_tasks):
    for b in range(number_of_tasks):
      couts+=y[(j,a,b)]*distances[a][b]/v
m.setObjective(couts, GRB.MINIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update()

# -- Affichage en mode texte du PL --
m.display()

# -- Résolution --
m.optimize()

# -- Vérification du statut et Affichage (le cas échéant) des solutions --
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tN'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tEST NON BORNÉ!!!")


# for j in range(number_of_tasks):
#   for i in range(number_of_ressources):
#     print(x[(j,i)].x,end=' ')
#   print("\n")

# print("\n\n\n")


# -- Affichage de la solution --
#print("La solution optimale est (d,q) = {} \navec pour valeur de l'objectif z = {}€".format((D.x, Q.x), m.objVal))
f=open("result.csv",'w')
res="taskId;performed;employeeName;startTime;\n"
for j in range(number_of_tasks):
  p=False
  for i in range(number_of_ressources):
    if x[(j,i)].x:
      res+="{};{};{};{};\n".format(L_tasks[j],1,L_ressources[i],s[j].x)
      p=True
  if not p:
    res+="{};;;;\n".format(L_tasks[j])
print(res)
f.write(res)
f.close()